# NDIS Database Preparation

## Geohazard & Infrastructure database compilation

This notebook documents the preprocessing pipeline for the Natural Disaster Information System (NDIS), a decision-support tool for planning RPAS (Remotely Piloted Aircraft Systems) missions in geohazard monitoring.

The preprocessing integrates six core datasets, each representing a distinct geohazard type:
1. Volcano
2. Landslide
3. Tsunami
4. Fault
5. Earthquake
6. Nuclear Power Plant

To ensure consistency across hazard types, raw datasets were parsed and harmonized into a unified schema. This involved:
- Converting spatial geometries to a common coordinate system
- Standardizing key fields such as intensity, duration, and economic loss
- Mapping hazard-specific attributes (e.g., magnitude, VEI) into shared analytical fields
- Handling missing values using severity-based proxies or simulation logic

Geospatial preprocessing steps include:
- Buffering and clipping to Exclusive Economic Zone (EEZ) boundaries
- Zonal statistics for population exposure within hazard zones
- Distance-based feature engineering (e.g., proximity to roads or coastlines)

All spatial analysis was performed in **ArcGIS Pro**, which also supported early-stage logic testing and scenario validation. The final decision logic, including RPAS and sensor recommendations, was implemented using a staggered rule-based workflow. Although machine learning models (e.g., decision trees) were evaluated, they were not used in the operational version due to limited generalizability.

This notebook serves as a reproducible reference for harmonized data preparation and logic integration in the NDIS system.

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

In [ ]:
%matplotlib inline
#ArcGIS packages
import arcpy
#from arcgis.mapping import WebScene
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from IPython.display import display
from arcgis.features import GeoAccessor
from arcgis import *
from arcpy.sa import Int
# Raster processing for dataframe
from rasterstats import zonal_stats
import rasterio

# basic packages
import csv
import numpy as np
import os
import timeit
import random
import string
from playsound import playsound
import gc # Force Garbage Collection. This helps reduce memory leaks in long loops.
import warnings

# Data management
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point  # to get points from long lat

# Request service
#from requests import Request
import json
import re
from functools import reduce
#from owslib.wfs import WebFeatureService
import sqlite3

# Plotting packages
import matplotlib.pyplot as plt
import seaborn as sns

# Get Datasets

## <font color='red'> 1. Volcano data </font>

<div class="alert alert-block alert-info">
<b>Note:</b>
    Citation: Global Volcanism Program, 2013. Volcanoes of the World, v. 4.11.0 (08 Jul 2022). Venzke, E (ed.). Smithsonian Institution. Downloaded 13 Jul 2022. https://doi.org/10.5479/si.GVP.VOTW4-2013.

Further info: https://volcano.si.edu/database/webservices.cfm

Service Layer: http://webservices.volcano.si.edu/geoserver/GVP-VOTW/ows?service=WFS&version=1.0.0&request=describefeaturetype&typeName=GVP-VOTW:E3WebApp_HoloceneVolcanoes
    
    Significant Volcano Eruption:
Citation: National Geophysical Data Center / World Data Service (NGDC/WDS): NCEI/WDS Global Significant Volcanic Eruptions Database. NOAA National Centers for Environmental Information. doi:10.7289/V5JW8BSH [8 July 2022]
</div>

In [ ]:
# Set the path to this geodatabase
gdb_path = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"  # Update the path accordingly

# Set the workspace to geodatabase
arcpy.env.workspace = gdb_path

# Specify the feature class name
feature_class_name = "volc84"
feature_class_path = f"{gdb_path}\\{feature_class_name}"

# Describe the feature class
desc = arcpy.Describe(feature_class_path)

# Check the spatial reference
spatial_reference = desc.spatialReference

# Print the spatial reference details
print(f"Spatial Reference of {feature_class_name}:")
print(f"  Name: {spatial_reference.name}")
print(f"  WKID: {spatial_reference.factoryCode}")
print(f"  WKT: {spatial_reference.exportToString()}")

In [ ]:
# Use arcpy to create a list of fields
fields = [f.name for f in arcpy.ListFields(f"{gdb_path}\\{feature_class_name}")]

# Use arcpy to create a search cursor and load the data into a list of dictionaries
data = []
with arcpy.da.SearchCursor(f"{gdb_path}\\{feature_class_name}", fields) as cursor:
    for row in cursor:
        data.append(dict(zip(fields, row)))

# Convert the list of dictionaries into a DataFrame
volc = pd.DataFrame(data)
volc.info()

In [ ]:
# Assuming 'Shape' column contains tuples (x, y), get the geometry for volcano database
volc['geometry'] = volc['Shape'].apply(lambda geom: Point(geom) if geom is not None else None)

In [ ]:
# Set the coordinate reference system (CRS) to WGS 84 (EPSG:4326)
#volc.set_crs(epsg=4326, inplace=True)
#print(volc.crs)

In [ ]:
# Create a GeoDataFrame
volc = gpd.GeoDataFrame(volc, geometry='geometry')
volc.info()

In [ ]:
# Extract column needed for the basic NDIS database to compile with other geohazards dataset.
# Volcano Number --> HazardID, longitude, latitude, HazardType (in numerical coded added after extraction). Volcano is 1.
vo_df = volc[['VolcanoNumber','LatitudeDecimal','LongitudeDecimal','geometry']].copy()
vo_df.rename(columns = {'VolcanoNumber':'HazardID','LatitudeDecimal':'latitude','LongitudeDecimal':'longitude'}, inplace = True)
vo_df['HazardType'] = 1
vo_df.info()

## <font color='red'> 2. Landslide Data </font>
Retrieved from NASA

Title: Global Landslide Catalog | Type: Feature Service | Owner: krolikie@unhcr.org_unhcr
https://maps.nccs.nasa.gov/arcgis/apps/MapAndAppGallery/index.html?appid=574f26408683485799d02e857e5d9521
<div class="alert alert-block alert-info">
<b>Note:</b>
    Citation: Kirschbaum, D.B., Stanley, T., & Zhou, Y. (2015). Spatial and temporal analysis of a global landslide catalog. Geomorphology, 249, 4-15. doi:10.1016/j.geomorph.2015.03.016

    Kirschbaum, D.B., Adler, R., Hong, Y., Hill, S., & Lerner-Lam, A. (2010). A global landslide catalog for hazard applications: method, results, and limitations. Natural Hazards, 52, 561-575. doi:10.1007/s11069-009-9401-4

Further info:
https://gpm.nasa.gov/landslides/data.html
</div>

In [ ]:
# After the file is downloaded, load the following from local path and check the GCS
ls_spatial_ref = arcpy.Describe(r"D:/NDIS_Database/04_Landslide/nasa_coolr_reports_point.shp").spatialReference
ls_spatial_ref

In [ ]:
# Read the shapefile from local disk
landslide_df = gpd.read_file(r"D:/NDIS_Database/04_Landslide/nasa_coolr_reports_point.shp")
landslide_df.info()

In [ ]:
# Extract column needed for the basic NDIS database to compile with other geohazards dataset.
# Volcano Number --> HazardID, longitude, latitude, HazardType (in numerical coded added after extraction). Volcano is 1.
ls_df = landslide_df[['ev_id','latitude','longitude', 'geometry']].copy()
ls_df.rename(columns = {'ev_id':'HazardID'}, inplace = True)
ls_df['HazardType'] = 2
ls_df.info()

In [ ]:
ls_df.crs

## <font color='red'> 3. Tsunami Data </font>

Data retrieved from NCEI NOAA - Global Historical Tsunami Database

<div class="alert alert-block alert-info">
<b>Note:</b>
    A feature layer displaying historical tsunami events from NCEI's Global Historical Tsunami Database. The Global Historical Tsunami Database consists of two related files containing information on tsunami events from 2000 B.C. to the present in the Atlantic, Indian, and Pacific Oceans; and the Mediterranean and Caribbean Seas.
    
    Citation: National Geophysical Data Center / World Data Service: NCEI/WDS Global Historical Tsunami Database. NOAA National Centers for Environmental Information. doi:10.7289/V5PN93H7 [4 August 2023]

Further info: https://ngdc.noaa.gov/hazard/hazards.shtml

Documentation: https://data.noaa.gov/metaview/page?xml=NOAA/NESDIS/NGDC/MGG/Hazards/iso/xml/G02151.xml&view=getDataView

Layer info: https://www.arcgis.com/home/item.html?id=5a44c3d4d465498993120b70ab568876
</div>

In [ ]:
# Specify the feature class name
feature_class_tsunami = "tsunami"  # Check for content pane
tsunami_path = f"{gdb_path}\\{feature_class_tsunami}"

# Describe the feature class
desc_tsun = arcpy.Describe(tsunami_path)

# Check the spatial reference
tsunami_sr = desc_tsun.spatialReference

# Print the spatial reference details
print(f"Spatial Reference of {feature_class_tsunami}:")
print(f"  Name: {tsunami_sr.name}")
print(f"  WKID: {tsunami_sr.factoryCode}")
print(f"  WKT:  {tsunami_sr.exportToString()}")

In [ ]:
# Use arcpy to create a list of fields
ts_fields = [f.name for f in arcpy.ListFields(f"{gdb_path}\\{feature_class_tsunami}")]

# Use arcpy to create a search cursor and load the data into a list of dictionaries
ts_data = []
with arcpy.da.SearchCursor(f"{gdb_path}\\{feature_class_tsunami}", ts_fields) as cursor:
    for row in cursor:
        ts_data.append(dict(zip(ts_fields, row)))

# Convert the list of dictionaries into a DataFrame
tsun = pd.DataFrame(ts_data)
tsun.info()

In [ ]:
ts_df = tsun[['ID','LONGITUDE', 'LATITUDE']].copy()
ts_df.rename(columns = {'ID':'HazardID','LONGITUDE':'longitude', 'LATITUDE':'latitude'}, inplace=True)
ts_df['HazardType'] = 3
ts_df.info()

In [ ]:
# Generate points geometry from longitude and latitude
ts_df['geometry'] = ts_df.apply(lambda x: Point(x['longitude'], x['latitude']), axis=1)

# Create a GeoDataFrame
ts_df = gpd.GeoDataFrame(ts_df, geometry='geometry')

# Set the coordinate reference system (CRS) to WGS 84 (EPSG:4326)
ts_df.set_crs(epsg=4326, inplace=True)

ts_df.crs

## <font color='red'> 4. Fault Data </font>

GEM Global Active Faults Database (GAF-DB)

<div class="alert alert-block alert-info">
<b>Note:</b>
    A feature layer displaying historical tsunami events from NCEI's Global Historical Tsunami Database. The Global Historical Tsunami Database consists of two related files containing information on tsunami events from 2000 B.C. to the present in the Atlantic, Indian, and Pacific Oceans; and the Mediterranean and Caribbean Seas.
    
    Citation: The GEM GAF-DB has been published in Earthquake Spectra.
    Styron, Richard, and Marco Pagani. “The GEM Global Active Faults Database.” Earthquake Spectra, vol. 36, no. 1_suppl, Oct. 2020, pp. 160–180, doi:10.1177/8755293020944182.

The link to the publication is here: https://journals.sagepub.com/doi/abs/10.1177/8755293020944182

Documentation: https://github.com/GEMScienceTools/gem-global-active-faults

</div>

In [ ]:
# Obtain Spatial Reference information for harmonized processing

af_spatial_ref = arcpy.Describe(r"D:/NDIS_Database/05_Fault/fault_points_wgs84.shp").spatialReference
af_spatial_ref

In [ ]:
# Prior to loading this file, fault data has been processes using Geoprocessing (via GUI) using Generate Points Along Lines tool.
# Read the shapefile from local disk
fault_df = gpd.read_file(r"D:/NDIS_Database/05_Fault/fault_points_wgs84.shp")
fault_df.info()

In [ ]:
# extract the longitude and latitude from the geometry field using the .x and .y attributes of the geometry column
if fault_df.geometry is not None:
    # Create new fields for longitude and latitude
    fault_df["longitude"] = fault_df.geometry.x
    fault_df["latitude"]  = fault_df.geometry.y

In [ ]:
faultdb = fault_df[['ORIG_FID', 'longitude','latitude', 'geometry']].copy()

### Assign ID

Add numeric prefix (177) and zero-padding the original index

Preserves traceability to original ORIG_FID.

In [ ]:
print(max(fault_df['ORIG_FID'].unique()))
print(min(fault_df['ORIG_FID'].unique()))

In [ ]:
fault_df = fault_df.copy()

# Use 7-digit global index padded with zeros → ensures 10-digit result with '177' prefix
fault_df["HazardID"] = fault_df.reset_index().index.map(lambda i: f"177{str(i).zfill(7)}")

# Check length and uniqueness
assert fault_df["HazardID"].str.len().max() == 10, "HazardID exceeds 10 digits!"
assert fault_df["HazardID"].is_unique, "HazardID values are not unique!"


In [ ]:
len(fault_df['HazardID'].unique())

In [ ]:
print(max(fault_df['HazardID'].unique()))
print(min(fault_df['HazardID'].unique()))

In [ ]:
# Mapping table to preserve tracability to ORIG_ID
fault_df[["HazardID", "ORIG_FID"]].to_csv(r"D:\NDIS_Database\05_Fault\fault_id_mapping.csv", index=False)

In [ ]:
# Extract column needed for the basic NDIS database to compile with other geohazards dataset.
# Volcano Number --> HazardID, longitude, latitude, HazardType (in numerical coded added after extraction). Volcano is 1.
faultdb = fault_df[['HazardID', 'longitude','latitude', 'geometry']].copy()
faultdb['HazardType'] = 4
faultdb.info()

## <font color='red'> 5. Earthquake Data </font>

### <font color='green'> Historcal part: </font>

### GHEC Catalog

GEM provides Global Historical Earthquake Catalogue (GHEC) from 1000 to 1903 (Albini, 2014). (https://platform.openquake.org/maps/80/download)

### SHEEC (SHARE European Earthquake Catalogue)

SHEEC catalogue covers the year 1000-1899 for Europe region specifically. It consists of data with magnitude ranges from 1.7 to 8.5 (Stucchi et al., 2012). The data could be downloaded at https: //www.emidius.eu/SHEEC/sheec_1000_1899.html.

### <font color='green'> Instrumental part: </font>

### ISC Bulletin/ISC Global
The ISC Bulletin has now been completely rebuilt for the period 1964-2010. As a result, the ISC hypocentre solutions and magnitudes for the entire period of 1964-latest are based on the ak135 velocity model and the location procedure that is currently used in operations.
(http://www.isc.ac.uk/iscbulletin/search/catalogue/)

The Bulletin of the International Seismological Centre relies on contributions from seismological agencies around the world. To date, a total of 573 agencies have contributed to the ISC Bulletin, throughout its history. For more information about the agency (http://www.isc.ac.uk/iscbulletin/agencies/).


### ISC-GEM Catalogue

The ISC-GEM Global Instrumental Earthquake Catalogue (1904-2016) is the result of a special effort to adapt and substantially extend and improve currently existing bulletin data for large earthquakes (magnitude 5.5 and above, plus continental events down to magnitude 5.0).
(http://www.isc.ac.uk/iscgem/)

### Load data from local layer file

In [ ]:
# Fix ID (identifier) of the earthquake events ID
# Load CSV files
df1 = pd.read_csv(r"D:\NDIS_Database\02_Earthquake\eq_1000_2023.csv") # Dataset with incorrect eventID
df2 = pd.read_csv(r"D:\NDIS_Database\02_Earthquake\GHEC1000_1903\eq1000.csv") # Dataset with correct GEHid

# Rename columns in df2 to match df1 (update the mapping if needed)
rename_mapping = {
    "Lat": "latitude",
    "Lon": "longitude",
    "Year": "year",
    "Mo": "month",
    "Da": "day"
}

df2.rename(columns=rename_mapping, inplace=True)

# Define matching fields
key_fields = ["longitude", "latitude", "year", "month", "day"]

# Merge on key fields
merged_df = df1.merge(df2[key_fields + ["GEHid"]], on=key_fields, how="left")

# Replace eventID with GEHid where a match is found
merged_df["eventID"] = merged_df["GEHid"].combine_first(merged_df["eventID"])

# Drop the temporary GEHid column
merged_df.drop(columns=["GEHid"], inplace=True)

merged_df.info()

In [ ]:
eq_df = merged_df[['eventID','longitude', 'latitude']].copy()
eq_df.rename(columns = {'eventID':'HazardID'}, inplace = True)
eq_df['HazardType'] = 5
eq_df.info()

In [ ]:
nan_in_eq = eq_df.isnull().sum().sum()

# printing the number of values present in
# the whole dataframe
print('Number of NaN values present: ' + str(nan_in_eq))

In [ ]:
# Generate points geometry from longitude and latitude
eq_df['geometry'] = eq_df.apply(lambda x: Point(x['longitude'], x['latitude']), axis=1)

# Create a GeoDataFrame
eq_df = gpd.GeoDataFrame(eq_df, geometry='geometry')

# Set the coordinate reference system (CRS) to WGS 84 (EPSG:4326)
eq_df.set_crs(epsg=4326, inplace=True)

eq_df.crs

-------------------------------------------------

# NEW ENTRY NUCLEAR POWER PLANT ☢️

In [ ]:
# Read data
nuclear_df = pd.read_csv(r"D:\NDIS_Database\15_NuclearPower\nuclearpp.csv")
nuclear_df.info()

In [ ]:
nuclear_df['GEM unit ID']

In [ ]:
# Assemble the cleaned nuclear hazard dataframe
cleaned_nuclear_df = pd.DataFrame({
    "HazardID": nuclear_df['GEM unit ID'],
    "latitude": pd.to_numeric(nuclear_df["Latitude"], errors="coerce"),
    "longitude": pd.to_numeric(nuclear_df["Longitude"], errors="coerce"),
    "HazardType": "Nuclear"
})

In [ ]:
cleaned_nuclear_df = cleaned_nuclear_df.copy()

# Extract last 6 digits of original G-code
cleaned_nuclear_df["unit_id_digits"] = cleaned_nuclear_df["HazardID"].str[-6:]

# Add prefix '1110' to make a uniform 10-digit numeric HazardID
cleaned_nuclear_df["HazardID"] = ("1110" + cleaned_nuclear_df["unit_id_digits"]).astype("int64")

# Optional: drop helper column
cleaned_nuclear_df.drop(columns=["unit_id_digits"], inplace=True)
cleaned_nuclear_df.info()

# Concatenate all data

In [ ]:
#Concatenate fault data to the rest of the datasets
ghz_df = pd.concat([vo_df, ls_df, ts_df, eq_df, faultdb, cleaned_nuclear_df]) #All Geohazards dataframe
ghz_df

In [ ]:
ghz_df.set_crs(epsg=4326, inplace=True)

In [ ]:
# Save locally in a supported format
output_path = "D:/NDIS_Database/ghz.gpkg"  # Adjust as needed
ghz_df.to_file(output_path, driver="GPKG")

In [ ]:
ghz_df.drop('geometry',axis=1).to_csv("D:/NDIS_Database/ghz84.csv")

In [ ]:
# For query purpose save it into sql
# Step 1: Create or open the SQLite database
conn = sqlite3.connect("ghz_data.sqlite")

# Step 2: Save the full multi-hazard DataFrame
ghz_df.to_sql("hazards", conn, if_exists="replace", index=False)

# Step 3: Close the connection
conn.close()

## <font color='red'> Countries EEZ Data </font>
Retrieved from Maritime Boundaries and Exclusive Economic Zones (200NM), version 12

https://www.marineregions.org/. https://doi.org/10.14284/632
<div class="alert alert-block alert-info">
<b>Citation:</b>
    Flanders Marine Institute (2024). Union of the ESRI Country shapefile and the Exclusive Economic Zones (version 4). Available online at https://www.marineregions.org/. https://doi.org/10.14284/698. Consulted on 2025-02-20
Further info:
https://www.marineregions.org/downloads.php#unioneezcountry
</div>

## Test Load

In [ ]:
ghz_gpkg = r"D:\NDIS_Database\ghz84.gpkg"
# Load the GeoPackage
ghz = gpd.read_file(ghz_gpkg, layer="ghz84")
ghz

In [ ]:
ghz.crs

----------------------------------------------------------------------------------

# <font color='red'> Clip and Near Analysis </font>

### CLIP and NEAR Analysis for the entire region

Divide the dataset into regions EEZ (country+ocean territory)

In [ ]:
# Read the shapefile from local disk
eez = gpd.read_file(r"D:\NDIS_Database\00_ClippingRegion\EEZ_land_union_v4_202410\EEZ_land_union_v4_202410.shp")
eez.info()

---

### EEZ Data Treatment

In [ ]:
# Set the path to this geodatabase
gdb_path = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"  # Update the path accordingly
gdb_path

---

In [ ]:
# Define feature classes
eez_country = "EEZ_country"
eez_union = "EEZ_union"

# Step 1: Generate Near Table
near_table = "EEZ_near_table"
if not arcpy.Exists(near_table):
    print("⚠️ Generating Near Table...")
    arcpy.GenerateNearTable_analysis(eez_union, eez_country, near_table, closest="CLOSEST", method="PLANAR")

# Step 2: Add NEAR_FID to EEZ_union
arcpy.JoinField_management(eez_union, "OBJECTID", near_table, "IN_FID", ["NEAR_FID"])

# Step 3: Merge EEZ_union into EEZ_country using NEAR_FID
with arcpy.da.UpdateCursor(eez_country, ["OBJECTID", "SHAPE@"]) as country_cursor:
    for country_row in country_cursor:
        country_id = country_row[0]
        country_geom = country_row[1]

        # Collect matching EEZ_union geometries
        union_geoms = []
        with arcpy.da.SearchCursor(eez_union, ["NEAR_FID", "SHAPE@"]) as union_cursor:
            for union_row in union_cursor:
                if union_row[0] == country_id and union_row[1] is not None:
                    union_geoms.append(union_row[1])

        # Merge all geometries if matches exist
        if union_geoms:
            merged_geom = reduce(lambda x, y: x.union(y), [country_geom] + union_geoms)
            country_row[1] = merged_geom
            country_cursor.updateRow(country_row)

print("✅ Finished merging polygons. EEZ_country now contains 275 polygons.")

In [ ]:
# Define feature classes
eez = "EEZ24"
# Field to check (change if needed)
unique_field = "FID"

# Get unique values
unique_values = set()
with arcpy.da.SearchCursor(eez, [unique_field]) as cursor:
    for row in cursor:
        if row[0]:  # Ignore Null values
            unique_values.add(row[0])

# Print results
print(f"✅ Unique {unique_field} count: {len(unique_values)}")
if len(unique_values) == 327:
    print("🎉 The field has exactly 327 unique values, matching the number of polygons!")
else:
    print(f"⚠️ Mismatch! Expected 327 but found {len(unique_values)}. Check for duplicates or missing values.")

In [ ]:
# Field to check
field_name = "ISO_SOV1"
search_value = "AAT"

# Check if "ANT" exists
exists = False
with arcpy.da.SearchCursor(eez, [field_name]) as cursor:
    for row in cursor:
        if row[0] == search_value:
            exists = True
            break  # No need to continue once found

# Print result
if exists:
    print(f"✅ '{search_value}' exists in the field '{field_name}'.")
else:
    print(f"❌ '{search_value}' was NOT found in '{field_name}'.")

In [ ]:
import random
import string

In [ ]:
input_eez = "EEZ24"

# Create a set to track used ISO_TER1 values and if NaN or duplicates, rename it from TERRITORY1 abbreviation
used_names = set()

# Function to generate a unique 3-letter code
def generate_unique_code(existing_names):
    while True:
        # Generate a random 3-letter combination
        code = ''.join(random.choices(string.ascii_uppercase, k=3))
        if code not in existing_names:  # Ensure it's unique
            return code

# Step 1: Update the ISO_TER1 values
with arcpy.da.UpdateCursor(input_eez, ['ISO_TER1', 'TERRITORY1']) as cursor:
    for row in cursor:
        iso_name = row[0]
        territory_name = row[1]

        # Check if this ISO_TER1 value is empty or not valid
        if not iso_name or len(iso_name) != 3 or not iso_name.isalpha():
            # Generate a unique 3-letter code
            unique_code = generate_unique_code(used_names)
            row[0] = unique_code  # Update ISO_TER1 value
            used_names.add(unique_code)  # Add to the set of used names

        # If it is valid and unique, keep the original value
        cursor.updateRow(row)

print("✅ ISO_TER1 values have been updated to unique 3-letter names.")

In [ ]:
from collections import defaultdict

# Set input dataset
input_eez = "EEZ"

# Track used ISO_TER1 values
used_names = set()

# Track duplicates: {ISO_TER1: [rows]}
iso_to_rows = defaultdict(list)

# Step 1: First Pass - Identify duplicates and invalid values
with arcpy.da.UpdateCursor(input_eez, ['ISO_TER1', 'UNION']) as cursor:
    for row in cursor:
        iso_name = row[0]
        territory_name = row[1]

        # Store existing values for duplication checks
        if iso_name and len(iso_name) == 3 and iso_name.isalpha():
            if iso_name in used_names:
                iso_to_rows[iso_name].append(row)  # Mark as duplicate
            else:
                used_names.add(iso_name)  # Add valid unique code
        else:
            iso_to_rows["INVALID"].append(row)  # Mark invalid values

# Function to generate a unique 3-letter code
def generate_unique_code(prefix, existing_names):
    while True:
        # Generate a code using the first 2 letters of UNION + 1 random letter
        if prefix and len(prefix) >= 2:
            code = prefix[:2].upper() + random.choice(string.ascii_uppercase)
        else:
            code = ''.join(random.choices(string.ascii_uppercase, k=3))

        if code not in existing_names:  # Ensure uniqueness
            return code

# Step 2: Second Pass - Fix duplicates and invalid values
with arcpy.da.UpdateCursor(input_eez, ['ISO_TER1', 'UNION']) as cursor:
    for row in cursor:
        iso_name = row[0]
        territory_name = row[1]

        # If this row was marked as a duplicate or invalid, update it
        if iso_name in iso_to_rows or iso_name == "INVALID":
            prefix = territory_name[:2] if territory_name else ""
            unique_code = generate_unique_code(prefix, used_names)
            row[0] = unique_code
            used_names.add(unique_code)

        cursor.updateRow(row)

print("✅ ISO_TER1 values updated: duplicates fixed, invalid values replaced.")

---------

### Test Using Smaller area

-----

In [ ]:
start_time = timeit.default_timer()
# Define paths
project_folder = r"D:\ArcGISProjects\GeohazardDB"
gdb_path = os.path.join(project_folder, "GeohazardDB.gdb")  # File Geodatabase

# Initialize a list to store processed geohazard datasets
ghz_list = []

# Input datasets
geohazard_layer = os.path.join(gdb_path, "g84_Clip")  # Geohazard dataset
road_layer = os.path.join(gdb_path, "grip_Clip")  # Global road dataset
country_layer = os.path.join(gdb_path, "eez3")  # Country boundaries

# Check if GDB exists, otherwise create it
if not arcpy.Exists(gdb_path):
    arcpy.CreateFileGDB_management(project_folder, "GeohazardDB.gdb")

# Iterate through each country
with arcpy.da.SearchCursor(country_layer, ["ISO_TER1", "SHAPE@"]) as country_cursor:
    for row in country_cursor:
        country_code = row[0]  # Country code (e.g., THA, USA)
        country_geometry = row[1]  # Country boundary geometry

        print(f"\u23F3 Processing country: {country_code}...")

        # Define output names inside the GDB
        ghz_clip = os.path.join(gdb_path, f"ghz_{country_code}")
        road_clip = os.path.join(gdb_path, f"road_{country_code}")
        near_output = os.path.join(gdb_path, f"near_{country_code}")

        # ---- Step 1: Clip Geohazard Data ----
        if arcpy.Exists(ghz_clip):
            arcpy.Delete_management(ghz_clip)
        try:
            arcpy.Clip_analysis(geohazard_layer, country_geometry, ghz_clip)
            print(f"  \u2705 Clipped geohazard layer: {ghz_clip}")
        except Exception as e:
            print(f"  \u274C Error clipping geohazard: {e}")
            continue  # Skip to next country if error occurs

        # ---- Step 2: Clip Road Data ----
        if arcpy.Exists(road_clip):
            arcpy.Delete_management(road_clip)
        try:
            arcpy.Clip_analysis(road_layer, country_geometry, road_clip)
            print(f"  \u2705 Clipped road layer: {road_clip}")
        except Exception as e:
            print(f"  {chr(0x274C)} Error clipping roads: {e}")
            continue  # Skip to next country if error occurs

        # ---- Step 3: Perform Near Analysis ----
        if arcpy.Exists(near_output):
            arcpy.Delete_management(near_output)
        try:
            arcpy.GenerateNearTable_analysis(
                in_features=ghz_clip,
                near_features=road_clip,
                out_table=near_output,
                search_radius="100000 Meters",  # Keep it in 100 Km
                location="LOCATION",  # Include X, Y coordinates
                angle="ANGLE",
                closest="CLOSEST",
                method="GEODESIC"
            )
            print(f"  \u2705 Near analysis completed: {near_output}")

        except Exception as e:
            print(f"  \u274C Error in Near Analysis: {e}")


        # ---- Step 4: Add distance to geohazard ----
        # Add "distance" field if it doesn't exist
        if "distance" not in [f.name for f in arcpy.ListFields(ghz_clip)]:
            arcpy.AddField_management(ghz_clip, "distance", "DOUBLE")

        # Update "distance" field with NEAR_DIST from the near table
        with arcpy.da.UpdateCursor(ghz_clip, ["OBJECTID", "distance"]) as ghz_cursor:
            for ghz_row in ghz_cursor:
                ghz_id = ghz_row[0]  # OBJECTID of ghz_XXX

                # Find matching NEAR_DIST from the near table
                with arcpy.da.SearchCursor(near_output, ["IN_FID", "NEAR_DIST"]) as near_cursor:
                    for near_row in near_cursor:
                        if near_row[0] == ghz_id:  # Match OBJECTID
                            ghz_row[1] = near_row[1]  # Assign NEAR_DIST
                            ghz_cursor.updateRow(ghz_row)
                            break  # Stop once found

        print(f"\u2705 NEAR_DIST added to {ghz_clip} as 'distance' field.")

        # Add "HazardID" field to near table if it doesn't exist
        if "HazardID" not in [f.name for f in arcpy.ListFields(near_output)]:
            arcpy.AddField_management(near_output, "HazardID", "TEXT")

        # Since this uses "CLOSEST" method, so it will only take 1 closest disance,
        # and discard the NEAR_FID as the 1:N will no longer needed. Instead it can be replaced with HazardID
        # to obtain the relationship with the ghz dataset
        # Use UpdateCursor to populate HazardID from ghz_XXX
        with arcpy.da.UpdateCursor(near_output, ["IN_FID", "HazardID"]) as cursor:
            for row in cursor:
                # Fetch the corresponding HazardID from ghz_XXX
                with arcpy.da.SearchCursor(ghz_clip, ["OBJECTID", "HazardID"]) as ghz_cursor:
                    for ghz_row in ghz_cursor:
                        if row[0] == ghz_row[0]:  # Match OBJECTID
                            row[1] = ghz_row[1]  # Assign HazardID
                            cursor.updateRow(row)
                            break  # Exit loop once matched

        print(f"\u2705 HazardID added to {near_output}, replacing NEAR_FID reference.")

        # Add processed dataset to list for final merge
        ghz_list.append(ghz_clip)

# Merge all processed ghz_XXX datasets into one: "ghz_dist"
ghz_dist = os.path.join(gdb_path, "ghz_dist")

if arcpy.Exists(ghz_dist):
    arcpy.Delete_management(ghz_dist)  # Ensure a fresh start

arcpy.Merge_management(ghz_list, ghz_dist)

print(f" All geohazard data merged into {ghz_dist} successfully!")

elapsed = timeit.default_timer() - start_time
print("\u2705 All processing completed! Elapsed time: %s minutes"%str(elapsed/60))

-----

# <h1 style="background-color:#d5f2e1; color: #bc6ee0;">Clip and Near Analysis Including Outside EEZ</h1>

---

In [ ]:
start_time = timeit.default_timer()

# Define paths
project_folder = r"D:\ArcGISProjects\GeohazardDB"
gdb_path = os.path.join(project_folder, "GeohazardDB.gdb")  # File Geodatabase

# Initialize lists to store processed geohazard datasets and near tables
ghz_list = []
near_tables = []
outside_eez_list = []  # List to store datasets with points outside EEZ

# Input datasets
geohazard_layer = os.path.join(gdb_path, "ghz84")  # Geohazard dataset
road_layer = os.path.join(gdb_path, "roads")  # Global road dataset
country_layer = os.path.join(gdb_path, "eez_country")  # Country boundaries

# Check if GDB exists, otherwise create it
if not arcpy.Exists(gdb_path):
    arcpy.CreateFileGDB_management(project_folder, "GeohazardDB.gdb")

# Get total number of countries to process
total_countries = int(arcpy.GetCount_management(country_layer)[0])

# Iterate through each country
with arcpy.da.SearchCursor(country_layer, ["ISO_TER1", "SHAPE@"]) as country_cursor:
    for index, row in enumerate(country_cursor, start=1):
        country_code = row[0]  # Country code (e.g., THA, USA)
        country_geometry = row[1]  # Country boundary geometry

        print(f"\u23F3 Processing country {index}/{total_countries}: {country_code}...")

        # Define output names inside the GDB
        ghz_clip = os.path.join(gdb_path, f"ghz_{country_code}")
        road_clip = os.path.join(gdb_path, f"road_{country_code}")
        near_output = os.path.join(gdb_path, f"near_{country_code}")

        # ---- Step 1: Clip Geohazard Data ----
        if arcpy.Exists(ghz_clip):
            arcpy.Delete_management(ghz_clip)
        try:
            arcpy.Clip_analysis(geohazard_layer, country_geometry, ghz_clip)
            print(f"  \u2705 Clipped geohazard layer: {ghz_clip}")
        except Exception as e:
            print(f"  \u274C Error clipping geohazard: {e}")
            continue  # Skip to next country if error occurs

        # ---- Step 2: Clip Road Data ----
        if arcpy.Exists(road_clip):
            arcpy.Delete_management(road_clip)
        try:
            arcpy.Clip_analysis(road_layer, country_geometry, road_clip)
            print(f"  \u2705 Clipped road layer: {road_clip}")
        except Exception as e:
            print(f"  {chr(0x274C)} Error clipping roads: {e}")
            continue  # Skip to next country if error occurs

        # ---- Step 3: Perform Near Analysis ----
        if arcpy.Exists(near_output):
            arcpy.Delete_management(near_output)
        try:
            arcpy.GenerateNearTable_analysis(
                in_features=ghz_clip,
                near_features=road_clip,
                out_table=near_output,
                search_radius="",  # No search radius
                location="LOCATION",  # Include X, Y coordinates
                angle="ANGLE",
                closest="CLOSEST",
                method="GEODESIC"
            )
            print(f"  \u2705 Near analysis completed: {near_output}")
            near_tables.append(near_output)  # Store near table

        except Exception as e:
            print(f"  \u274C Error in Near Analysis: {e}")

        # ---- Step 4: Add distance to geohazard ----
        # Add "distance" field if it doesn't exist
        if "distance" not in [f.name for f in arcpy.ListFields(ghz_clip)]:
            arcpy.AddField_management(ghz_clip, "distance", "DOUBLE")

        # Update "distance" field with NEAR_DIST from the near table
        with arcpy.da.UpdateCursor(ghz_clip, ["OBJECTID", "distance"]) as ghz_cursor:
            for ghz_row in ghz_cursor:
                ghz_id = ghz_row[0]  # OBJECTID of ghz_XXX

                # Find matching NEAR_DIST from the near table
                found_match = False
                with arcpy.da.SearchCursor(near_output, ["IN_FID", "NEAR_DIST"]) as near_cursor:
                    for near_row in near_cursor:
                        if near_row[0] == ghz_id:  # Match OBJECTID
                            ghz_row[1] = near_row[1]  # Assign NEAR_DIST
                            ghz_cursor.updateRow(ghz_row)
                            found_match = True
                            break  # Stop once found

                if not found_match:  # If no match found, add to outside_eez list
                    outside_eez_list.append(ghz_clip)

        print(f"\u2705 NEAR_DIST added to {ghz_clip} as 'distance' field.")

        # Add "HazardID" field to near table if it doesn't exist
        if "HazardID" not in [f.name for f in arcpy.ListFields(near_output)]:
            arcpy.AddField_management(near_output, "HazardID", "TEXT")

        # Use UpdateCursor to populate HazardID from ghz_XXX
        with arcpy.da.UpdateCursor(near_output, ["IN_FID", "HazardID"]) as cursor:
            for row in cursor:
                # Fetch the corresponding HazardID from ghz_XXX
                with arcpy.da.SearchCursor(ghz_clip, ["OBJECTID", "HazardID"]) as ghz_cursor:
                    for ghz_row in ghz_cursor:
                        if row[0] == ghz_row[0]:  # Match OBJECTID
                            row[1] = ghz_row[1]  # Assign HazardID
                            cursor.updateRow(row)
                            break  # Exit loop once matched

        print(f"\u2705 HazardID added to {near_output}, replacing NEAR_FID reference.")

        # Add processed dataset to list for final merge
        ghz_list.append(ghz_clip)

# Merge all processed ghz_XXX datasets into one: "ghz_dist"
ghz_dist = os.path.join(gdb_path, "ghz_dist")

if arcpy.Exists(ghz_dist):
    arcpy.Delete_management(ghz_dist)  # Ensure a fresh start

arcpy.Merge_management(ghz_list, ghz_dist)

print(f" All geohazard data merged into {ghz_dist} successfully!")

# ---- Step 5: Compile all near tables into one ----
compiled_near_table = os.path.join(gdb_path, "compiled_near_table")
if arcpy.Exists(compiled_near_table):
    arcpy.Delete_management(compiled_near_table)

# Create empty table to store results
arcpy.CreateTable_management(gdb_path, "compiled_near_table")
arcpy.AddField_management(compiled_near_table, "FROM_X", "DOUBLE")
arcpy.AddField_management(compiled_near_table, "FROM_Y", "DOUBLE")
arcpy.AddField_management(compiled_near_table, "NEAR_X", "DOUBLE")
arcpy.AddField_management(compiled_near_table, "NEAR_Y", "DOUBLE")
arcpy.AddField_management(compiled_near_table, "NEAR_FID", "LONG")
arcpy.AddField_management(compiled_near_table, "HazardID", "TEXT")

# Insert cursor for compiled near table
with arcpy.da.InsertCursor(compiled_near_table, ["FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y", "NEAR_FID", "HazardID"]) as insert_cursor:
    for near_table in near_tables:
        with arcpy.da.SearchCursor(near_table, ["FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y", "NEAR_FID", "HazardID"]) as cursor:
            for row in cursor:
                insert_cursor.insertRow(row)

print(f"\u270 All near tables compiled into {compiled_near_table} successfully!")

# ---- Step 6: Handle points outside EEZ ----
outside_eez_output = os.path.join(gdb_path, "outside_eez")

# Try deleting the feature class if it exists (avoid locking issues)
if arcpy.Exists(outside_eez_output):
    try:
        arcpy.Delete_management(outside_eez_output)
        print(f"\u270 Deleted existing {outside_eez_output}")
    except Exception as e:
        print(f"&#9888 Error deleting {outside_eez_output}: {e}")
        time.sleep(2)  # Wait a bit and retry
        arcpy.Delete_management(outside_eez_output)

# Get all field names from geohazard_layer (excluding OBJECTID & Shape)
fields = [f.name for f in arcpy.ListFields(geohazard_layer) if f.type not in ("OID", "Geometry")]
fields.insert(0, "SHAPE@")  # Ensure geometry is included

# Create a new feature class with the same schema
try:
    arcpy.CreateFeatureclass_management(
        gdb_path, "outside_eez", "POINT",
        template=geohazard_layer,  # Preserve schema
        spatial_reference=arcpy.Describe(geohazard_layer).spatialReference
    )
    print("\u270 Successfully created outside_eez feature class")
except Exception as e:
    print(f"\u274C Failed to create outside_eez: {e}")

# ---- Step 7: Create Polyline Feature Class from Compiled Near Table ----
polyline_output = os.path.join(gdb_path, "compiled_near_lines")
if arcpy.Exists(polyline_output):
    arcpy.Delete_management(polyline_output)

# Use XY To Line tool to create lines from compiled near table
arcpy.XYToLine_management(
    compiled_near_table,
    polyline_output,
    "FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y",
    "",  # Optional Line ID field
)

print(f"\u270 Polylines created from compiled near table at {polyline_output}!")


elapsed = timeit.default_timer() - start_time
print("\u2705 All processing completed! Elapsed time: %s minutes" % str(elapsed / 60))

In [ ]:
# ---- Step 6: Handle points outside EEZ ----
outside_eez_output = os.path.join(gdb_path, "outside_eez")

# Try deleting the feature class if it exists (avoid locking issues)
if arcpy.Exists(outside_eez_output):
    try:
        arcpy.Delete_management(outside_eez_output)
        print(f"✅ Deleted existing {outside_eez_output}")
    except Exception as e:
        print(f"⚠️ Error deleting {outside_eez_output}: {e}")
        time.sleep(2)  # Wait a bit and retry
        arcpy.Delete_management(outside_eez_output)

# Get all field names from geohazard_layer (excluding OBJECTID & Shape)
fields = [f.name for f in arcpy.ListFields(geohazard_layer) if f.type not in ("OID", "Geometry")]
fields.insert(0, "SHAPE@")  # Ensure geometry is included

# Create a new feature class with the same schema
try:
    arcpy.CreateFeatureclass_management(
        gdb_path, "outside_eez", "POINT",
        template=geohazard_layer,  # Preserve schema
        spatial_reference=arcpy.Describe(geohazard_layer).spatialReference
    )
    print("✅ Successfully created outside_eez feature class")
except Exception as e:
    print(f"❌ Failed to create outside_eez: {e}")

In [ ]:
# ---- Step 7: Create Polyline Feature Class from Compiled Near Table ----
polyline_output = os.path.join(gdb_path, "compiled_near_lines")
if arcpy.Exists(polyline_output):
    arcpy.Delete_management(polyline_output)

# Use XY To Line tool to create lines from compiled near table
arcpy.XYToLine_management(
    compiled_near_table,
    polyline_output,
    "FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y",
    "",  # Optional Line ID field
)

print(f"✅ Polylines created from compiled near table at {polyline_output}!")

In [ ]:

# Paths
source_gdb = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"
output_gdb = r"D:\ArcGISProjects\GeohazardDB\NDIS.gdb"

# Make sure output GDB exists
if not arcpy.Exists(output_gdb):
    arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

# Example test country codes — replace or expand this list as needed
test_country_codes = ["THA", "IDN", "JPN"]

for country_code in test_country_codes:
    ghz_fc = os.path.join(source_gdb, f"ghz_{country_code}")
    road_fc = os.path.join(source_gdb, f"road_{country_code}")
    near_table = os.path.join(output_gdb, f"near_{country_code}")
    point_output = os.path.join(output_gdb, f"near_points_{country_code}")

    print(f"\n--- Processing {country_code} ---")

    # Check if input features exist
    if not arcpy.Exists(ghz_fc) or not arcpy.Exists(road_fc):
        print(f"  ⛔ Skipped: Missing clipped layers for {country_code}")
        continue

    try:
        # Step 1: Generate Near Table
        if arcpy.Exists(near_table):
            arcpy.Delete_management(near_table)
        arcpy.GenerateNearTable_analysis(
            in_features=ghz_fc,
            near_features=road_fc,
            out_table=near_table,
            location="LOCATION",
            angle="ANGLE",
            closest="CLOSEST",
            method="GEODESIC"
        )
        print(f"  ✅ Near table created: {near_table}")
    except Exception as e:
        print(f"  ❌ Error generating near table: {e}")
        continue

    try:
        # Step 2: Convert XY to Point
        if arcpy.Exists(point_output):
            arcpy.Delete_management(point_output)
        arcpy.XYTableToPoint_management(
            in_table=near_table,
            out_feature_class=point_output,
            x_field="NEAR_X",
            y_field="NEAR_Y",
            coordinate_system=arcpy.Describe(ghz_fc).spatialReference
        )
        print(f"  ✅ XY to Point completed: {point_output}")
    except Exception as e:
        print(f"  ❌ Error converting to point: {e}")

In [ ]:
start_time = timeit.default_timer()
# Paths
source_gdb = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"
output_gdb = r"D:\ArcGISProjects\GeohazardDB\NDIS.gdb"

# Field schema for compiled table
fields = [
    ("FROM_X", "DOUBLE"),
    ("FROM_Y", "DOUBLE"),
    ("NEAR_X", "DOUBLE"),
    ("NEAR_Y", "DOUBLE"),
    ("NEAR_FID", "LONG"),
    ("HazardID", "TEXT")
]

# Create output GDB if needed
if not arcpy.Exists(output_gdb):
    arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

# Prepare compiled near table
compiled_table = os.path.join(output_gdb, "compiled_near_table")
if arcpy.Exists(compiled_table):
    arcpy.Delete_management(compiled_table)
arcpy.CreateTable_management(output_gdb, "compiled_near_table")
for name, ftype in fields:
    arcpy.AddField_management(compiled_table, name, ftype)

# Load all country codes from EEZ feature class
eez_fc = os.path.join(source_gdb, "eez_country")
country_codes = [row[0] for row in arcpy.da.SearchCursor(eez_fc, ["ISO_TER1"])]

# Loop through all countries
for code in country_codes:
    ghz_fc = os.path.join(source_gdb, f"ghz_{code}")
    road_fc = os.path.join(source_gdb, f"road_{code}")
    near_table = os.path.join(output_gdb, f"near_{code}")

    print(f"\U0001f373 Processing {code}...")

    if not arcpy.Exists(ghz_fc) or not arcpy.Exists(road_fc):
        print(f"  ⛔ Missing input for {code}, skipping.")
        continue

    try:
        # Generate Near Table
        if arcpy.Exists(near_table):
            arcpy.Delete_management(near_table)
        arcpy.GenerateNearTable_analysis(
            in_features=ghz_fc,
            near_features=road_fc,
            out_table=near_table,
            location="LOCATION",
            angle="ANGLE",
            closest="CLOSEST",
            method="GEODESIC"
        )
        print(f"  ✅ Near table created: {near_table}")
    except Exception as e:
        print(f"  ❌ Error generating near table for {code}: {e}")
        continue

    try:
        # Append rows to compiled table
        with arcpy.da.InsertCursor(compiled_table, [f[0] for f in fields]) as insert_cursor:
            with arcpy.da.SearchCursor(near_table, [f[0] for f in fields]) as read_cursor:
                for row in read_cursor:
                    insert_cursor.insertRow(row)
        print(f"  ➕ Appended {code} to compiled table.")
    except Exception as e:
        print(f"  ❌ Error appending rows for {code}: {e}")
        continue

# Final step: XY To Line from compiled table
polyline_output = os.path.join(output_gdb, "compiled_near_lines")
if arcpy.Exists(polyline_output):
    arcpy.Delete_management(polyline_output)

try:
    arcpy.XYToLine_management(
        in_table=compiled_table,
        out_feature_class=polyline_output,
        startx_field="FROM_X",
        starty_field="FROM_Y",
        endx_field="NEAR_X",
        endy_field="NEAR_Y",
        line_type="GEODESIC"
    )
    print(f"\n✅ Global polyline layer created: {polyline_output}")
except Exception as e:
    print(f"\n❌ Error creating polyline layer: {e}")


elapsed = timeit.default_timer() - start_time
print("\u2705 All processing completed! Elapsed time: %s minutes" % str(elapsed / 60))

In [ ]:
compiled_lines = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\compiled_near_lines"

# Add new field for length in meters
if "length_m" not in [f.name for f in arcpy.ListFields(compiled_lines)]:
    arcpy.AddField_management(compiled_lines, "length_m", "DOUBLE")

# Calculate geodesic length in meters
arcpy.CalculateGeometryAttributes_management(
    in_features=compiled_lines,
    geometry_property=[["distance_m", "LENGTH_GEODESIC"]],
    length_unit="METERS"
)
print("✅ length_m field populated with geodesic length in meters.")


In [ ]:
compiled_table = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb\compiled_near_table"
arcpy.JoinField_management(
    in_data=compiled_lines,
    in_field="OID",
    join_table=compiled_table,
    join_field="OBJECTID",  # the join key
    fields=["HazardID"]
)
print("✅ HazardID injected into compiled_near_lines.")

----

# Population

In [ ]:
ghz_pap = pd.read_csv(r"D:\NDIS_Database\ghz_paper.csv")
ghz_pap.info()

In [ ]:
start_time = timeit.default_timer()
# Parameters
population_raster = r"D:\NDIS_Database\12_Population_synthetic\nasapopct.tif"
buffer_dist = 30000  # 30 km
chunk_size = 10000

output_dir = r"D:\NDIS_Database\zonal_chunks"

# Check if it's a file, not a folder
if os.path.exists(output_dir) and not os.path.isdir(output_dir):
    os.remove(output_dir)

# Now safely create the directory
os.makedirs(output_dir, exist_ok=True)

# Convert DataFrame to GeoDataFrame
ghz_pap["geometry"] = ghz_pap.apply(
    lambda row: Point(row["longitude"], row["latitude"]), axis=1
)
input_gdf = gpd.GeoDataFrame(ghz_pap, geometry="geometry", crs="EPSG:4326")

# Buffer in degrees (WGS84 safe approx)
buffer_deg = buffer_dist / 111320.0

# Tracking
total = len(input_gdf)
print(f"🔹 Total features: {total}")
final_chunks = []
suspicious_chunks = []

for start in range(0, total, chunk_size):
    end = min(start + chunk_size, total)
    print(f"⏳ Processing chunk {start+1} to {end}")

    chunk = input_gdf.iloc[start:end].copy()

    # Suppress geographic CRS buffer warning
    warnings.filterwarnings("ignore", message="Geometry is in a geographic CRS.*")
    chunk["geometry"] = chunk.geometry.buffer(buffer_deg)

    # Drop bad geometries
    chunk = chunk[chunk["geometry"].notnull()]
    chunk = chunk[chunk.is_valid]

    if chunk.empty:
        print(f"⚠️ Skipping empty/invalid chunk {start+1} to {end}")
        continue

    # Compute zonal stats with overflow suppression
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        try:
            stats = zonal_stats(
                chunk,
                population_raster,
                stats=["sum"],
                geojson_out=False,
                nodata=-9999  # Or None if unknown
            )
        except Exception as e:
            print(f"❌ Error in chunk {start+1} to {end}: {e}")
            continue

    if not stats or all(row.get("sum") is None for row in stats):
        print(f"⚠️ No stats found for chunk {start+1} to {end}")
        chunk["pop"] = 0
    else:
        chunk["pop"] = [float(row.get("sum", 0) or 0) for row in stats]

    # Flag suspicious high-pop zones
    chunk["pop_flagged"] = chunk["pop"].apply(lambda x: "⚠️ check" if x > 20_000_000 else "")
    max_pop = chunk["pop"].max()
    if max_pop > 20_000_000:
        print(f"⚠️ High population in chunk {start+1}-{end}: {max_pop:,.0f}")
        suspicious_chunks.append(chunk[chunk["pop"] > 20_000_000].copy())

    # Save chunk to CSV immediately
    chunk_out_path = os.path.join(output_dir, f"chunk_{start+1}_{end}.csv")
    chunk.drop(columns="geometry").to_csv(chunk_out_path, index=False)
    print(f"📁 Saved chunk to: {chunk_out_path}")

    print(f"✅ Chunk {start+1} to {end} processed.")
    # Append to final result in memory
    final_chunks.append(chunk.copy())  # keep geometry for now

    # Save chunk to disk (crash recovery)
    chunk_out_path = os.path.join(output_dir, f"chunk_{start+1}_{end}.csv")
    chunk.drop(columns="geometry").to_csv(chunk_out_path, index=False)

    # Memory cleanup
    del chunk, stats
    gc.collect()

# Save suspicious rows summary
if suspicious_chunks:
    suspicious_df = pd.concat(suspicious_chunks, ignore_index=True)
    suspicious_df.to_csv(os.path.join(output_dir, "suspicious_population_zones.csv"), index=False)
    print(f"🟡 Saved {len(suspicious_df)} suspicious rows for manual review.")
else:
    print("✅ No suspicious population zones found.")

print("🎉 All done.")


# Combine all chunks in memory
final_gdf = pd.concat(final_chunks, ignore_index=True)
print("✅ Combined all chunks into final_gdf")

final_gdf.to_csv(r"D:\NDIS_Database\ghz_pop.csv", index=False)

elapsed = timeit.default_timer() - start_time
print("\u2705 All processing completed! Elapsed time: %s minutes"%str(elapsed/60))

In [ ]:
suspicious_df.info()

In [ ]:
# Step 1: Reload or reuse original input GeoDataFrame
chunk = input_gdf.iloc[1930000:1938350].copy()  # Python is 0-based

# Step 2: Buffer and clean geometry
warnings.filterwarnings("ignore", message="Geometry is in a geographic CRS.*")
chunk["geometry"] = chunk.geometry.buffer(buffer_deg)
chunk = chunk[chunk["geometry"].notnull()]
chunk = chunk[~chunk["geometry"].is_empty]
chunk = chunk[chunk.is_valid]

# Step 3: Rerun zonal stats safely
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    stats = zonal_stats(
        chunk,
        population_raster,
        stats=["sum"],
        geojson_out=False,
        nodata=-9999
    )

# Step 4: Fill population values
chunk["pop"] = [float(row.get("sum", 0) or 0) for row in stats]
chunk["pop_flagged"] = chunk["pop"].apply(lambda x: "⚠️ check" if x > 20_000_000 else "")

# Step 5: Save the corrected chunk
chunk_out_path = os.path.join(output_dir, "chunk_1930001_1938350.csv")
chunk.drop(columns="geometry").to_csv(chunk_out_path, index=False)
print(f"✅ Patched chunk saved to {chunk_out_path}")

# Optional: append to final in-memory result
#final_chunks.append(chunk.copy())


In [ ]:
# ArcGIS ObjectID starts at 1, but pandas index starts at 0
# So subtract 1
pandas_row_ids = [i - 1 for i in null_geom_ids]

# Isolate the rows from final_gdf
null_rows = final_gdf.iloc[pandas_row_ids].copy()

In [ ]:
print(min(null_rows.longitude))
print(max(null_rows.longitude))
print(min(null_rows.latitude))
print(max(null_rows.latitude))

In [ ]:
# List of ArcGIS-reported IDs
arcgis_oids = [
    1045, 14943, 14999, 15003, 15037, 15052, 15108, 18189, 64503, 72741,
    99537, 102208, 103016, 105219, 114251, 114516, 115189, 116052, 116932,
    116991, 117447, 117792, 118519, 121364, 139022, 355328, 359478, 370280,
    370413, 371531, 386363, 386367, 386387, 388549, 389557, 390083, 393224,
    393645, 394422, 394774, 403304, 404977, 476193, 539023, 867092, 895514,
    927888, 987286, 988385, 1110501, 1112965, 1112970, 1163972, 1178798,
    1183599, 1289453, 1289586, 1338653, 1340317, 1340620, 1520597, 1527977,
    1527982, 1582745, 1585801, 1619277, 1620106, 1621234, 1622439, 1630938,
    1633828, 1633854, 1648844, 1821680, 1888743, 1888745, 1888748, 1888750,
    1888752, 1888754, 1888926, 1903415, 1903418, 1903419, 1903420
]

# Extract suspicious rows
sus_rows = final_gdf.iloc[arcgis_oids]

In [ ]:
at_origin_coords = final_gdf[
    (final_gdf["longitude"] == 0.0) & (final_gdf["latitude"] == 0.0)
]
at_origin_coords

In [ ]:
at_origin_coords.info()

In [ ]:
invalid_coords = final_gdf[
    (final_gdf["latitude"] > 90) | (final_gdf["latitude"] < -90) |
    (final_gdf["longitude"] > 180) | (final_gdf["longitude"] < -180)
]

In [ ]:
# Step 1: Get the final chunk (assuming input_gdf is still in memory)
final_chunk = input_gdf.iloc[1930000:1938350].copy()

# Step 2: Extract rows with inf population from final_gdf
inf_rows = final_gdf[np.isinf(final_gdf["pop"])].copy()

# Step 3: Drop geometry and rebuild clean GeoDataFrame
inf_rows = inf_rows.drop(columns="geometry", errors="ignore")
inf_rows["geometry"] = inf_rows.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)
inf_gdf = gpd.GeoDataFrame(inf_rows, geometry="geometry", crs="EPSG:4326")

# Step 4: Combine both sets (resetting index not strictly necessary but keeps things tidy)
redo_gdf = pd.concat([final_chunk, inf_gdf], ignore_index=True)

print(f"🔁 Total to reprocess: {len(redo_gdf)} rows")

In [ ]:
null_or_empty_geom_df = ghz_pap[ghz_pap["latitude"].isna() | ghz_pap["longitude"].isna()]


# Tsunami Reprocess

In [ ]:
# Select only the desired columns from merged_df_final
tsun_2concat = tsunami[[
    "UNIQUE_ID",
    "LATITUDE",
    "LONGITUDE",
    "intensity",
    "economic_loss_million",
    "duration_minutes"
]].copy()
tsun_2concat.info()

In [ ]:
# Rename fields to match geohazard schema
tsun_2concat = tsunami[[
    "UNIQUE_ID",
    "LATITUDE",
    "LONGITUDE",
    "intensity",
    "economic_loss_million",
    "duration_minutes"
]].copy()

# Rename columns
tsun_2concat.rename(columns={
    "UNIQUE_ID": "HazardID",
    "LATITUDE": "latitude",
    "LONGITUDE": "longitude"
}, inplace=True)
tsun_2concat.info()

In [ ]:
# Save to CSV for ArcGIS processing
csv_path = "D:/ArcGISProjects/GeohazardDB/tsunami_standardized.csv"
tsun_2concat.to_csv(csv_path, index=False)

tsun_2concat.head()

In [ ]:
# Paths
gdb_path = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"
country_layer = os.path.join(gdb_path, "eez_country")
road_layer = os.path.join(gdb_path, "roads")  # Assuming the full global road dataset is here
output_clip = os.path.join(gdb_path, "road_MNE")

# Get geometry of MNE
where_clause = "ISO_TER1 = 'MNE'"

# Temporary layer
arcpy.MakeFeatureLayer_management(country_layer, "country_lyr", where_clause)

# Clip road layer
arcpy.Clip_analysis(
    in_features=road_layer,
    clip_features="country_lyr",
    out_feature_class=output_clip
)

print(f"✅ Clipped road_MNE saved at: {output_clip}")

In [ ]:
start_time = timeit.default_timer()

# Paths
project_folder = r"D:\ArcGISProjects\GeohazardDB"
input_csv = os.path.join(project_folder, "tsunami_standardized.csv")
ndis_gdb = os.path.join(project_folder, "NDIS.gdb")
road_gdb = os.path.join(project_folder, "GeohazardDB.gdb")
road_layer_template = os.path.join(road_gdb, "roads")
country_layer = os.path.join(project_folder, "GeohazardDB.gdb", "eez_country")

# Convert CSV to point layer
tsun_layer = os.path.join(ndis_gdb, "tsun_input")
arcpy.management.XYTableToPoint(
    in_table=input_csv,
    out_feature_class=tsun_layer,
    x_field="longitude",
    y_field="latitude",
    coordinate_system=arcpy.SpatialReference(4326)
)

# Near analysis preparation
ghz_list = []
near_tables = []

total_countries = int(arcpy.GetCount_management(country_layer)[0])

with arcpy.da.SearchCursor(country_layer, ["ISO_TER1", "SHAPE@"]) as country_cursor:
    for index, row in enumerate(country_cursor, start=1):
        iso = row[0]
        shape = row[1]
        print(f"⏳ Processing {iso} ({index}/{total_countries})...")

        tsun_clip = os.path.join(ndis_gdb, f"tsun_{iso}")
        road_clip = os.path.join(road_gdb, f"road_{iso}")
        near_table = os.path.join(ndis_gdb, f"near_{iso}")

        # Clip tsunami points
        if arcpy.Exists(tsun_clip):
            arcpy.Delete_management(tsun_clip)
        arcpy.analysis.Clip(tsun_layer, shape, tsun_clip)

        # Run Near (roads already pre-clipped)
        if arcpy.Exists(near_table):
            arcpy.Delete_management(near_table)
        arcpy.analysis.GenerateNearTable(
            in_features=tsun_clip,
            near_features=road_clip,
            out_table=near_table,
            location="LOCATION",
            angle="ANGLE",
            closest="CLOSEST",
            method="GEODESIC"
        )
        print(f"  ✅ Near table created: {near_table}")
        near_tables.append(near_table)

        # Add 'distance' to tsun_clip
        if "distance" not in [f.name for f in arcpy.ListFields(tsun_clip)]:
            arcpy.AddField_management(tsun_clip, "distance", "DOUBLE")

        with arcpy.da.UpdateCursor(tsun_clip, ["OBJECTID", "distance"]) as up_cursor:
            for up_row in up_cursor:
                oid = up_row[0]
                with arcpy.da.SearchCursor(near_table, ["IN_FID", "NEAR_DIST"]) as near_cursor:
                    for near_row in near_cursor:
                        if near_row[0] == oid:
                            up_row[1] = near_row[1]
                            up_cursor.updateRow(up_row)
                            break

        # Add 'HazardID' to near table
        if "HazardID" not in [f.name for f in arcpy.ListFields(near_table)]:
            arcpy.AddField_management(near_table, "HazardID", "TEXT")

        with arcpy.da.UpdateCursor(near_table, ["IN_FID", "HazardID"]) as cursor:
            for row in cursor:
                with arcpy.da.SearchCursor(tsun_clip, ["OBJECTID", "HazardID"]) as src:
                    for src_row in src:
                        if row[0] == src_row[0]:
                            row[1] = src_row[1]
                            cursor.updateRow(row)
                            break

        ghz_list.append(tsun_clip)

# Merge tsunami point results
merged_output = os.path.join(ndis_gdb, "tsun_dist")
if arcpy.Exists(merged_output):
    arcpy.Delete_management(merged_output)
arcpy.Merge_management(ghz_list, merged_output)
print(f"✅ All tsunami points merged into {merged_output}")

# Merge near tables
compiled_near_table = os.path.join(ndis_gdb, "compiled_near_table_tsun")
if arcpy.Exists(compiled_near_table):
    arcpy.Delete_management(compiled_near_table)

arcpy.CreateTable_management(ndis_gdb, "compiled_near_table_tsun")
for field in [("FROM_X", "DOUBLE"), ("FROM_Y", "DOUBLE"), ("NEAR_X", "DOUBLE"),
              ("NEAR_Y", "DOUBLE"), ("NEAR_FID", "LONG"), ("HazardID", "TEXT")]:
    arcpy.AddField_management(compiled_near_table, field[0], field[1])

with arcpy.da.InsertCursor(compiled_near_table, ["FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y", "NEAR_FID", "HazardID"]) as insert_cursor:
    for table in near_tables:
        with arcpy.da.SearchCursor(table, ["FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y", "NEAR_FID", "HazardID"]) as cursor:
            for row in cursor:
                insert_cursor.insertRow(row)

# Create line layer
line_fc = os.path.join(ndis_gdb, "compiled_near_lines_tsun")
if arcpy.Exists(line_fc):
    arcpy.Delete_management(line_fc)

arcpy.XYToLine_management(
    compiled_near_table, line_fc,
    "FROM_X", "FROM_Y", "NEAR_X", "NEAR_Y"
)
print(f"✅ Lines created: {line_fc}")

elapsed = timeit.default_timer() - start_time
print(f"✅ All tsunami near analysis completed in {elapsed/60:.2f} minutes")


In [ ]:
tsun_fixed = gpd.read_file(r"D:\ArcGISProjects\GeohazardDB\NDIS.gdb", layer="tsun_dist")

# Step 1: Drop existing tsunami rows from ghz_pap
ghz_no_tsun = ghz_pap[ghz_pap["HazardType"] != "Tsunami"].copy()

In [ ]:
tsun_fixed["HazardType"] = "Tsunami"

In [ ]:
# Step 3: Concatenate both
ghz_all = pd.concat([ghz_no_tsun, tsun_fixed], ignore_index=True)

In [ ]:
# Check for NaN values in the "distance" field
nan_rows = ghz_all[ghz_all["distance"].isna()]

# Display count by HazardType (if field exists)
if "HazardType" in nan_rows.columns:
    print(nan_rows["HazardType"].value_counts())
else:
    print("HazardType column not found in the data.")

# Optionally: show how many total
print(f"Total rows with NaN in distance: {len(nan_rows)}")

In [ ]:
ghz_all.info()

In [ ]:
# Step 4: Clean up temporary columns
ghz_all.drop(columns=["cpm_total_time", "travel_time"], inplace=True)
ghz_all.info()

In [ ]:
ghz_all.to_csv(r"D:\NDIS_Database\ghz_paper2.csv", index=False)

In [ ]:
import arcpy
from arcpy.sa import *

arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"D:\NDIS_Database\12_Population_synthetic"

# Input raster
input_raster = "nasapopct.tif"
output_raster = "nasapopct_int.tif"

# Convert float to int (rounded)
int_ras = Int(Raster(input_raster) + 0.5)  # round to nearest integer

# Save to file
int_ras.save(output_raster)

print(f"✅ Saved integer raster to: {output_raster}")

In [ ]:
# Confirm type
r = arcpy.Raster(r"D:\NDIS_Database\12_Population_synthetic\nasapopct_int.tif")
print(r.pixelType)  # Should now be 'S32' or similar

In [ ]:
# === SETUP ===
arcpy.CheckOutExtension("Spatial")

# Paths and constants
project_folder = r"D:\ArcGISProjects\GeohazardDB"
gdb_ghz = os.path.join(project_folder, "GeohazardDB.gdb")
gdb_ndis = os.path.join(project_folder, "NDIS.gdb")
country_layer = os.path.join(gdb_ghz, "eez_country")
population_raster = r"D:\NDIS_Database\12_Population_synthetic\nasapopct_int.tif"
output_field = "pop_sum"
log_path = os.path.join(project_folder, "pop_stat_log.txt")

# Load as Raster to ensure integer pixel type is respected
value_raster = arcpy.sa.Int(arcpy.Raster(population_raster))
print(f"✅ Forcing Int raster (wrapped): {value_raster.pixelType}")

# Extract ISO codes from country layer
iso_codes = set()
with arcpy.da.SearchCursor(country_layer, ["ISO_TER1"]) as cursor:
    for row in cursor:
        iso = row[0]
        if iso:
            iso_codes.add(iso)

# Find feature classes matching the ISO suffixes
def get_matching_fcs(gdb_path, prefix):
    arcpy.env.workspace = gdb_path
    return [fc for fc in arcpy.ListFeatureClasses(f"{prefix}_*") if fc.split("_")[-1] in iso_codes]

ghz_fc = get_matching_fcs(gdb_ghz, "ghz")
nuc_fc = get_matching_fcs(gdb_ghz, "nuc")
tsun_fc = get_matching_fcs(gdb_ndis, "tsun")
all_fc = ghz_fc + nuc_fc + tsun_fc

print(f"Total matching FCs: {len(all_fc)}")

# === ZONAL STATISTICS LOOP ===
start_total = time.time()

with open(log_path, "w", encoding="utf-8") as log:
    for idx, fc in enumerate(all_fc, start=1):
        start_time = time.time()
        try:
            print(f"⏳ [{idx}/{len(all_fc)}] Processing: {fc}")
            log.write(f"[START] {fc}\n")

            # Determine full path
            if fc in tsun_fc:
                fc_path = os.path.join(gdb_ndis, fc)
            else:
                fc_path = os.path.join(gdb_ghz, fc)

            # Add population field if missing
            field_names = [f.name for f in arcpy.ListFields(fc_path)]
            if output_field not in field_names:
                arcpy.AddField_management(fc_path, output_field, "DOUBLE")

            # Zonal statistics as table
            zone_table = os.path.join("in_memory", f"{fc}_zonal")
            arcpy.sa.ZonalStatisticsAsTable(
                in_zone_data=fc_path,
                zone_field="OBJECTID",
                in_value_raster=value_raster,
                out_table=zone_table,
                ignore_nodata="DATA",
                statistics_type="SUM"
            )

            # Join and transfer values
            arcpy.JoinField_management(
                in_data=fc_path,
                in_field="OBJECTID",
                join_table=zone_table,
                join_field="OBJECTID",
                fields=["SUM"]
            )

            if "SUM" in [f.name for f in arcpy.ListFields(fc_path)]:
                with arcpy.da.UpdateCursor(fc_path, ["SUM", output_field]) as cursor:
                    for row in cursor:
                        row[1] = row[0]
                        cursor.updateRow(row)
                arcpy.DeleteField_management(fc_path, ["SUM"])

            elapsed = round(time.time() - start_time, 2)
            log.write(f"[OK] {fc} completed in {elapsed} sec\n")
            print(f"✅ Done in {elapsed} sec")

        except Exception as e:
            log.write(f"[FAIL] {fc} error: {str(e)}\n")
            print(f"❌ Error in {fc}: {e}")

        # Clear in_memory and force garbage collection
        arcpy.Delete_management("in_memory")
        gc.collect()

    total_elapsed = round(time.time() - start_total, 2)
    log.write(f"\n✅ All done in {total_elapsed / 60:.2f} min\n")
    print(f"\n✅ All done in {total_elapsed / 60:.2f} min")


----------------

# Statistics of Database

In [ ]:
# Set the path to this geodatabase
gdb_path = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"  # This gdb path

In [ ]:
# Set the path to this geodatabase
gdb_path = r"D:\ArcGISProjects\GeohazardDB\GeohazardDB.gdb"  # This gdb path
# Specify the feature class name
ghz_dist = "ghz_dist"  # Geohazard feature class
ghz_path = f"{gdb_path}\\{ghz_dist}"

# Use arcpy to create a list of fields
ghz_fields = [f.name for f in arcpy.ListFields(f"{gdb_path}\\{ghz_dist}")]

# Use arcpy to create a search cursor and load the data into a list of dictionaries
ghz_data = []
with arcpy.da.SearchCursor(f"{gdb_path}\\{ghz_dist}", ghz_fields) as cursor:
    for row in cursor:
        ghz_data.append(dict(zip(ghz_fields, row)))

# Convert the list of dictionaries into a DataFrame
ghzdf = pd.DataFrame(ghz_data)
ghzdf.info()

In [ ]:
nan_in_ghz = ghzdf.isnull().sum().sum()

# printing the number of values present in
# the whole dataframe
print('Number of NaN values present: ' + str(nan_in_ghz))

In [ ]:
geohazard_layer = os.path.join(gdb_path, "ghz_dist")

# Define the output feature class
output_fc = os.path.join(gdb_path, "cleaned_geohazard_data")

# Retrieve data from feature class and store it in DataFrame
fields = ["OBJECTID", "Shape@", "HazardID", "latitude", "longitude", "HazardType", "distance"]
data = []

# Use SearchCursor to extract data from feature class
with arcpy.da.SearchCursor(geohazard_layer, fields) as cursor:
    for row in cursor:
        # Separate geometry from the non-geometry fields
        geometry = row[1]  # Shape@ is at index 1
        data.append(row[:1] + (geometry,) + row[2:])  # Append geometry separately

# Create DataFrame without geometry
df = pd.DataFrame(data, columns=["OBJECTID", "Shape@", "HazardID", "latitude", "longitude", "HazardType", "distance"])

# Convert 'distance' field to numeric, handling errors as NaN
df['distance'] = pd.to_numeric(df['distance'], errors='coerce')

# Filter out rows with NaN distance values
df_no_nulls = df[df['distance'].notna()]

# Handle duplicates by HazardID (keep row with smallest distance)
df_no_nulls = df_no_nulls.loc[df_no_nulls.groupby('HazardID')['distance'].idxmin()]

# Create the output feature class by keeping the geometry
arcpy.management.CreateFeatureclass(
    out_path=gdb_path,
    out_name="cleaned_geohazard_data",
    geometry_type="POINT",  # Change to 'POINT' if the geometry is a point
    template=geohazard_layer,  # Copy schema from the original feature class
    spatial_reference=arcpy.Describe(geohazard_layer).spatialReference
)

# Insert cleaned data into the new feature class using InsertCursor
with arcpy.da.InsertCursor(output_fc, ["SHAPE@", "HazardID", "latitude", "longitude", "HazardType", "distance"]) as cursor:
    for idx, row in df_no_nulls.iterrows():
        geometry = row['Shape@']  # Retrieve geometry separately
        cursor.insertRow([geometry, row['HazardID'], row['latitude'], row['longitude'], row['HazardType'], row['distance']])

print(f"Cleaned data written to: {output_fc}")
# Play sound to notify that it's done
# playsound(r"C:\Windows\Media\Alarm09.wav")

In [ ]:
ghz_pop = pd.read_csv(r"D:\NDIS_Database\ghz_pop.csv")
ghz_pop.info()

In [ ]:
# Find rows where pop is inf, -inf, or NaN
mask = ~np.isfinite(ghz_pop["pop"])
problematic_rows = ghz_pop[mask]

print(f"Total problematic rows: {len(problematic_rows)}")

-----------------------
# Population Handle inf
--------------------------

In [ ]:
ghz_pap = pd.read_csv(r"D:\NDIS_Database\ghz_paper2.csv")
ghz_pap.info()

In [ ]:

with rasterio.open(r"D:/NDIS_Database/12_Population_synthetic/nasapopct.tif") as src:
    print("CRS:", src.crs)
    print("Bounds:", src.bounds)
    print("Res:", src.res)
    print("NoData:", src.nodata)
    print("Dtype:", src.dtypes[0])

In [ ]:
src_path = r"D:/NDIS_Database/12_Population_synthetic/nasapopct.tif"

with rasterio.open(src_path) as src:
    data = src.read(1)

    print("CRS:", src.crs)
    print("Bounds:", src.bounds)
    print("Res:", src.res)
    print("NoData:", src.nodata)
    print("Dtype:", src.dtypes[0])

    # Check value counts
    nodata_val = src.nodata
    print("\nValue stats:")
    print("  NaNs:", np.isnan(data).sum())
    print("  Infs:", np.isinf(data).sum())
    print("  NoData:", np.sum(data == nodata_val))
    print("  Zeros:", np.sum(data == 0))
    print("  Valid (>0):", np.sum(data > 0))

In [ ]:
dst_path = r"D:/NDIS_Database/12_Population_synthetic/nasapopct_fixed_int.tif"

with rasterio.open(src_path) as src:
    profile = src.profile
    data = src.read(1)

    # Replace extreme NoData and other invalids
    data[data == src.nodata] = 0
    data[~np.isfinite(data)] = 0

    # Convert to integer
    int_data = np.round(data).astype("int32")
    profile.update(dtype="int32", nodata=0)

    with rasterio.open(dst_path, "w", **profile) as dst:
        dst.write(int_data, 1)

print("✅ Saved cleaned raster to:", dst_path)

In [ ]:
src_path = r"D:/NDIS_Database/12_Population_synthetic/nasapopct.tif"
dst_path = r"D:/NDIS_Database/12_Population_synthetic/nasapopct_int32.tif"

with rasterio.open(src_path) as src:
    profile = src.profile
    data = src.read(1)

    # Clean invalids
    data[~np.isfinite(data)] = 0
    data[data == src.nodata] = 0

    # to integer
    int_data = np.round(data).astype("int32")
    profile.update(dtype='int32', nodata=0)

    with rasterio.open(dst_path, 'w', **profile) as dst:
        dst.write(int_data, 1)

print("✅ Saved raster with int32 precision:", dst_path)

In [ ]:
start_time = timeit.default_timer()

# === PARAMETERS ===
population_raster = r"D:\NDIS_Database\12_Population_synthetic\nasapopct_int32.tif"
buffer_dist = 30000  # 30 km
chunk_size = 10000
output_dir = "zonal_chunks"
os.makedirs(output_dir, exist_ok=True)

# === LOAD AND PREP ===
ghz_pap["geometry"] = ghz_pap.apply(
    lambda row: Point(row["longitude"], row["latitude"]), axis=1
)
input_gdf = gpd.GeoDataFrame(ghz_pap, geometry="geometry", crs="EPSG:4326")

buffer_deg = buffer_dist / 111320.0
total = len(input_gdf)
print(f"🔹 Total features: {total}")

suspicious_chunks = []
nan_rows_all = []

# === PROCESS IN CHUNKS ===
for start in range(0, total, chunk_size):
    end = min(start + chunk_size, total)
    print(f"⏳ Processing chunk {start+1} to {end}")

    chunk = input_gdf.iloc[start:end].copy()

    # Suppress geographic CRS buffer warning
    warnings.filterwarnings("ignore", message="Geometry is in a geographic CRS.*")
    chunk["geometry"] = chunk.geometry.buffer(buffer_deg)

    # Drop bad geometries
    chunk = chunk[chunk["geometry"].notnull()]
    chunk = chunk[chunk.is_valid]

    if chunk.empty:
        print(f"⚠️ Skipping empty/invalid chunk {start+1} to {end}")
        continue

    # Zonal stats with correct nodata and pixel inclusion
    try:
        stats = zonal_stats(
            chunk,
            population_raster,
            stats=["sum"],
            geojson_out=False,
            nodata=-3.4028235e+38,
            all_touched=True
        )
    except Exception as e:
        print(f"❌ Error in chunk {start+1} to {end}: {e}")
        continue

    # Clean and assign population values
    pop_vals = []
    nan_rows = []
    for i, row in enumerate(stats):
        val = row.get("sum", 0)
        if val is None or not np.isfinite(val):
            pop_vals.append(np.nan)
            nan_rows.append(i)
        else:
            pop_vals.append(val)

    chunk["pop"] = pop_vals

    # Flag very high values
    chunk["pop_flagged"] = chunk["pop"].apply(
        lambda x: "⚠️ check" if pd.notna(x) and x > 20_000_000 else ""
    )

    max_pop = chunk["pop"].max()
    if pd.notna(max_pop) and max_pop > 20_000_000:
        print(f"⚠️ High population in chunk {start+1}-{end}: {max_pop:,.0f}")
        suspicious_chunks.append(chunk[chunk["pop"] > 20_000_000].copy())

    # Log NaN results for QA
    if nan_rows:
        nan_rows_all.append(chunk.iloc[nan_rows][["HazardID", "latitude", "longitude", "pop"]].copy())

    # Save each chunk
    chunk_out_path = os.path.join(output_dir, f"chunk_{start+1}_{end}.csv")
    chunk.drop(columns="geometry").to_csv(chunk_out_path, index=False)
    print(f"📁 Saved chunk to: {chunk_out_path}")

    # Clean up memory
    del chunk, stats
    gc.collect()
    print(f"✅ Chunk {start+1} to {end} processed.")

# === FINAL OUTPUT ===

# Save suspicious rows with very high population
if suspicious_chunks:
    suspicious_df = pd.concat(suspicious_chunks, ignore_index=True)
    suspicious_df.to_csv(os.path.join(output_dir, "suspicious_population_zones.csv"), index=False)
    print(f"🟡 Saved {len(suspicious_df)} suspicious rows for manual review.")
else:
    print("✅ No suspicious population zones found.")

# Save rows with NaN population values
if nan_rows_all:
    nan_df = pd.concat(nan_rows_all, ignore_index=True)
    nan_df.to_csv(os.path.join(output_dir, "nan_population_zones.csv"), index=False)
    print(f"🟠 Saved {len(nan_df)} NaN population rows for inspection.")
else:
    print("✅ No NaN rows in final result.")

print("🎉 All done.")
elapsed = timeit.default_timer() - start_time
print(f"✅ All tsunami near analysis completed in {elapsed/60:.2f} minutes")

In [ ]:
# Combine all chunk CSVs into a single DataFrame
chunk_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.startswith("chunk_") and f.endswith(".csv")]
final_df = pd.concat([pd.read_csv(f) for f in chunk_files], ignore_index=True)
print(f"🧩 Combined all {len(chunk_files)} chunk CSVs into final_df with {len(final_df):,} rows.")
final_df.to_csv(os.path.join(output_dir, "all_chunks_combined.csv"), index=False)
print("💾 Saved final combined CSV.")

In [ ]:
final_df.to_csv(r"D:\NDIS_Database\zonal_chunks\all_chunks_combined.csv", index=False)

In [ ]:
final_df.HazardType.unique()

In [ ]:
suspicious_df.to_csv(r"D:\NDIS_Database\zonal_chunks\suspicious_population.csv", index=False)

In [ ]:
final_df = pd.read_csv(r"D:\NDIS_Database\zonal_chunks\all_chunks_combined.csv")
final_df.info()